In [ ]:
!pip install transformers datasets evaluate

In [ ]:
!pip install accelerate -U

In [3]:
import accelerate
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [4]:
from huggingface_hub import notebook_login

notebook_login()

### Word form transmission

In [5]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')

In [ ]:
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test.drop('span', axis = 1)

In [7]:
train.loc[train['label'] != 'Word form transmission', 'label'] = 0
train.loc[train['label'] == 'Word form transmission', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0][0:700]])
test.loc[test['label'] != 'Word form transmission', 'label'] = 0
test.loc[test['label'] == 'Word form transmission', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0][0:200]])

In [8]:
train = Dataset.from_pandas(train)
test = Dataset.from_pandas(test)

In [9]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [10]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [11]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1316 [00:00<?, ? examples/s]

Map:   0%|          | 0/361 [00:00<?, ? examples/s]

In [12]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [13]:
accuracy = evaluate.load("f1")

In [14]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [15]:
id2label = {0: "Other", 1: "WFT"}
label2id = {"Other": 0, "WFT": 1}

In [16]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [19]:
training_args = TrainingArguments(
    output_dir="L1-classifier-WFT",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,No log,0.381536,0.772563
2,No log,0.339960,0.857143


TrainOutput(global_step=166, training_loss=0.24226655155779367, metrics={'train_runtime': 28.9934, 'train_samples_per_second': 90.779, 'train_steps_per_second': 5.725, 'total_flos': 48993274373232.0, 'train_loss': 0.24226655155779367, 'epoch': 2.0})

In [20]:
trainer.push_to_hub()

events.out.tfevents.1712803132.46f740a342e3.1129.1:   0%|          | 0.00/5.63k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-WFT/commit/6e122e9011546c3ac2ade2638f73fbb561d9095e', commit_message='End of training', commit_description='', oid='6e122e9011546c3ac2ade2638f73fbb561d9095e', pr_url=None, pr_revision=None, pr_num=None)

### Tense semantics

In [ ]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test.drop('span', axis = 1)

In [58]:
train.loc[train['label'] != 'Tense semantics', 'label'] = 0
train.loc[train['label'] == 'Tense semantics', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0][0:700]])
test.loc[test['label'] != 'Tense semantics', 'label'] = 0
test.loc[test['label'] == 'Tense semantics', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0][0:200]])

In [59]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [61]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [62]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [63]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1277 [00:00<?, ? examples/s]

Map:   0%|          | 0/349 [00:00<?, ? examples/s]

In [64]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [54]:
accuracy = evaluate.load("accuracy")

In [65]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [66]:
id2label = {0: "Other", 1: "TenSem"}
label2id = {"Other": 0, "TenSem": 1}

In [68]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [69]:
training_args = TrainingArguments(
    output_dir="L1-classifier-TenSem",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.282226,0.893983
2,No log,0.265986,0.902579


TrainOutput(global_step=160, training_loss=0.30676167011260985, metrics={'train_runtime': 31.6753, 'train_samples_per_second': 80.631, 'train_steps_per_second': 5.051, 'total_flos': 58149048403776.0, 'train_loss': 0.30676167011260985, 'epoch': 2.0})

In [70]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1712803895.46f740a342e3.1129.4:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-TenSem/commit/282c0912de79bec29c2cfd5cc0fcd0eb450e91cf', commit_message='End of training', commit_description='', oid='282c0912de79bec29c2cfd5cc0fcd0eb450e91cf', pr_url=None, pr_revision=None, pr_num=None)

### Synonyms

In [ ]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test.drop('span', axis = 1)

In [72]:
train.loc[train['label'] != 'Synonyms', 'label'] = 0
train.loc[train['label'] == 'Synonyms', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0][0:700]])
test.loc[test['label'] != 'Synonyms', 'label'] = 0
test.loc[test['label'] == 'Synonyms', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0][0:200]])

In [75]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [76]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [77]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [78]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1168 [00:00<?, ? examples/s]

Map:   0%|          | 0/321 [00:00<?, ? examples/s]

In [79]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [80]:
accuracy = evaluate.load("accuracy")

In [81]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [85]:
id2label = {0: "Other", 1: "Syn"}
label2id = {"Other": 0, "Syn": 1}

In [86]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
training_args = TrainingArguments(
    output_dir="L1-classifier-Synonyms",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.619957,0.654206
2,No log,0.578146,0.682243


TrainOutput(global_step=146, training_loss=0.592991345549283, metrics={'train_runtime': 29.3899, 'train_samples_per_second': 79.483, 'train_steps_per_second': 4.968, 'total_flos': 43515540458496.0, 'train_loss': 0.592991345549283, 'epoch': 2.0})

In [88]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

events.out.tfevents.1712804119.46f740a342e3.1129.6:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-Synonyms/commit/4493b775f533bfb27becdbd977d10fdc7d1d36ff', commit_message='End of training', commit_description='', oid='4493b775f533bfb27becdbd977d10fdc7d1d36ff', pr_url=None, pr_revision=None, pr_num=None)

### Copying expression

In [89]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test.drop('span', axis = 1)

,label,text
0,Word form transmission,The statistic above were compiled from individ...
1,Word form transmission,The chain drive provides another bit of reduci...
2,Word form transmission,The aftermaths of the game saw several lineup ...
3,Word form transmission,comparing with other red dots.
4,Word form transmission,So cast iron is approximately four percents ca...
...,...,...
721,Word form transmission,A key relationships are that with elected poli...
722,Copying expression,Cos it set down on our corner shop.
723,Tense semantics,The table demonstrates the info regarding the ...
724,Word form transmission,These savings constitute direct reducings in p...


In [90]:
train.loc[train['label'] != 'Copying expression', 'label'] = 0
train.loc[train['label'] == 'Copying expression', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0][0:700]])
test.loc[test['label'] != 'Copying expression', 'label'] = 0
test.loc[test['label'] == 'Copying expression', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0][0:200]])

In [91]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [92]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [93]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [94]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1193 [00:00<?, ? examples/s]

Map:   0%|          | 0/307 [00:00<?, ? examples/s]

In [95]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [96]:
accuracy = evaluate.load("accuracy")

In [97]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [99]:
id2label = {0: "Other", 1: "CopExp"}
label2id = {"Other": 0, "CopExp": 1}

In [100]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [101]:
training_args = TrainingArguments(
    output_dir="L1-classifier-CopExp",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.459520,0.804560
2,No log,0.360500,0.866450


TrainOutput(global_step=150, training_loss=0.49094696044921876, metrics={'train_runtime': 27.6782, 'train_samples_per_second': 86.205, 'train_steps_per_second': 5.419, 'total_flos': 44446951855296.0, 'train_loss': 0.49094696044921876, 'epoch': 2.0})

In [102]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

events.out.tfevents.1712804259.46f740a342e3.1129.7:   0%|          | 0.00/5.58k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-CopExp/commit/d92126bef9af34424a5f21e65a89e69a86bfa134', commit_message='End of training', commit_description='', oid='d92126bef9af34424a5f21e65a89e69a86bfa134', pr_url=None, pr_revision=None, pr_num=None)

### Transliteration

In [103]:
train = pd.read_csv('/content/real_train.tsv', sep='\t')
test = pd.read_csv('/content/real_dev.tsv', sep='\t')
train = train.rename(columns={'Copying expression':'label', 'set down':'span', 'It set down on the Moon in 1970 .': 'text'})
train = train.drop('span', axis = 1)
test = test.rename(columns={'Word form transmission':'label', 'declining':'span', 'Several factors were linked to the declining .': 'text'})
test.drop('span', axis = 1)

,label,text
0,Word form transmission,The statistic above were compiled from individ...
1,Word form transmission,The chain drive provides another bit of reduci...
2,Word form transmission,The aftermaths of the game saw several lineup ...
3,Word form transmission,comparing with other red dots.
4,Word form transmission,So cast iron is approximately four percents ca...
...,...,...
721,Word form transmission,A key relationships are that with elected poli...
722,Copying expression,Cos it set down on our corner shop.
723,Tense semantics,The table demonstrates the info regarding the ...
724,Word form transmission,These savings constitute direct reducings in p...


In [104]:
train.loc[train['label'] != 'Transliteration', 'label'] = 0
train.loc[train['label'] == 'Transliteration', 'label'] = 1
train = pd.concat([train.loc[train['label'] == 1], train.loc[train['label'] == 0][0:700]])
test.loc[test['label'] != 'Transliteration', 'label'] = 0
test.loc[test['label'] == 'Transliteration', 'label'] = 1
test = pd.concat([test.loc[test['label'] == 1], test.loc[test['label'] == 0][0:200]])

In [105]:
train = Dataset.from_pandas(train.sample(frac=1))
test = Dataset.from_pandas(test.sample(frac=1))

In [106]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-cased")

In [107]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, truncation=True)

In [108]:
tokenized_train = train.map(preprocess_function, batched=True)
tokenized_test = test.map(preprocess_function, batched=True)

Map:   0%|          | 0/1452 [00:00<?, ? examples/s]

Map:   0%|          | 0/388 [00:00<?, ? examples/s]

In [109]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [110]:
accuracy = evaluate.load("accuracy")

In [111]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [112]:
id2label = {0: "Other", 1: "Transliteration"}
label2id = {"Other": 0, "Transliteration": 1}

In [113]:
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-cased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [114]:
training_args = TrainingArguments(
    output_dir="L1-classifier-Transliteration",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.252844,0.907216
2,No log,0.209990,0.922680


TrainOutput(global_step=182, training_loss=0.32654565245240597, metrics={'train_runtime': 32.3304, 'train_samples_per_second': 89.822, 'train_steps_per_second': 5.629, 'total_flos': 56350389506400.0, 'train_loss': 0.32654565245240597, 'epoch': 2.0})

In [115]:
trainer.push_to_hub()

events.out.tfevents.1712804367.46f740a342e3.1129.8:   0%|          | 0.00/5.63k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Zlovoblachko/L1-classifier-Transliteration/commit/0ca6d2d66c72173617d27bf915493dc25de6067a', commit_message='End of training', commit_description='', oid='0ca6d2d66c72173617d27bf915493dc25de6067a', pr_url=None, pr_revision=None, pr_num=None)